# Assignment 1 - CIC-1205

## Exercise 2 - Diamond price prediction

Student: Balthazar Paixão


## Code


In [37]:
import numpy as np
import pandas as pd

In [38]:
df_diamonds = pd.read_csv("../class-repo/cic1205/data/diamonds.csv")

In [39]:
df_diamonds.sample(3)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
47361,47362,0.54,Ideal,H,VVS1,61.8,55.0,1846,5.21,5.24,3.23
35859,35860,0.40,Ideal,F,VS1,62.4,53.0,917,4.72,4.76,2.96
34892,34893,0.31,Ideal,D,VS2,61.8,56.0,879,4.39,4.35,2.70


In [40]:
df_diamonds = df_diamonds.drop(columns=["Unnamed: 0"])

In [41]:
colnames = [column for column in df_diamonds.columns]
colnames

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y', 'z']

## Pré-processamento dos dados


Esta etapa é muito importante, uma má execução desta etapa pode causar problemas no treino.

Aqui, verificamos os valores presentes em colunas que sabemos que se tratam de dados categóricos. Então é necessário pensar se estes dados são _nominais_ (valor que indica categoria, sem levar em consideração um "ranking") ou _ordinais_ (valor que indica categoria, mas que carrega uma relação de ordem com demais valores presentes).

Podemos ver que os dados são ordinais. Podemos ver que há valores para os quais vai haver um corte, cor ou claridade do diamante que vai ser ideal ou não, então isso é modelado na codificação.


In [42]:
print(df_diamonds["cut"].unique())
print(df_diamonds["color"].unique())
print(df_diamonds["clarity"].unique())

['Ideal' 'Premium' 'Good' 'Very Good' 'Fair']
['E' 'I' 'J' 'H' 'F' 'G' 'D']
['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF']


In [43]:
df_diamonds["cut"].replace(
    {"Fair": 1, "Good": 2, "Very Good": 3, "Premium": 4, "Ideal": 5}, inplace=True
)
df_diamonds["color"].replace(
    {"J": 1, "I": 2, "H": 3, "G": 4, "F": 5, "E": 6, "D": 7}, inplace=True
)
df_diamonds["clarity"].replace(
    {"I1": 1, "SI2": 2, "SI1": 3, "VS2": 4,
        "VS1": 5, "VVS2": 6, "VVS1": 7, "IF": 8},
    inplace=True,
)

In [44]:
print(df_diamonds["cut"].unique())
print(df_diamonds["color"].unique())
print(df_diamonds["clarity"].unique())

[5 4 2 3 1]
[6 2 1 3 5 4 7]
[2 3 5 4 6 7 1 8]


In [45]:
df_diamonds.sample(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
38148,0.50,3,2,5,63.1,56.0,1013,5.07,5.01,3.18
13922,1.01,1,4,4,67.8,59.0,5666,6.07,6.02,4.10
10843,1.06,4,3,3,62.8,58.0,4872,6.49,6.47,4.07


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [47]:
# Divisão entre treino e teste
X = df_diamonds.drop(columns=['price'])
y = df_diamonds['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Normalização das features
É importante normalizarmos as features para equilibrar as suas ordens de grandeza, para os mesmos serem processados na etapa de treino. Isso é feito porque a existência de discrepâncias entres essas orders de grandeza prejudica o ajuste dos modelos em alguns algoritmo de aprendizado, os que dependem de cálculos de distância.

É também importante entender que o ajuste do *scaler* deve ser feito apenas sobre o conjunto de treino,
para evitar que haja vazamento de dados (fenômeno conhecido como [data leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning))).

In [54]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(43152, 9)
(43152,)
(10788, 9)
(10788,)


## Treinamento do modelo

In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [57]:
LR = LinearRegression()
LR.fit(X_train, y_train)

LS = Lasso()
LS.fit(X_train, y_train)

DT = DecisionTreeRegressor()
DT.fit(X_train, y_train)

RF = RandomForestRegressor()
RF.fit(X_train, y_train)

KNN = KNeighborsRegressor()
KNN.fit(X_train, y_train)

GB = GradientBoostingRegressor()
GB.fit(X_train, y_train)

GradientBoostingRegressor()

In [58]:
#Como métricas de avaliação, use o MSE e o coeficiente de determinação R2.

In [60]:
#Linear Regression
y_pred = LR.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("Linear Regression")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 1448045.115154087
R2: 0.9092783543017693


In [61]:
#LASSO
y_pred = LS.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("LASSO")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 1448780.1497121027
R2: 0.9092323035647789


In [62]:
#Decision Tree
y_pred = DT.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("Decision Tree")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 576896.984705228
R2: 0.9638567588101452


In [63]:
#Random Forest
y_pred = RF.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("Random Forest")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 310225.53772970056
R2: 0.9805640231606582


In [65]:
#K Nearest Neighbors
y_pred = KNN.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("K Nearest Neighbors")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 511276.9869373378
R2: 0.9679679250479345


In [66]:
#Gradient Boosting
y_pred = GB.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
r2 = r2_score(y_true=y_test, y_pred=y_pred)

print("Gradient Boosting")
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 384786.0184953325
R2: 0.9758927256656283
